### Creating the Dataloader
##### Dataloader process slightly adapted from our lecture.
###### RandomSplitter: Dividing into training and validation sets. 
###### item_tfms and batch_tfms: image augmentation and presizing.

In [ ]:
# defining the datablock
satellite_block = DataBlock(
    blocks=(ImageBlock, MaskBlock(codes=dictionary_landuse)),
    get_items=get_image_files,
    get_y=get_mask,
    splitter=RandomSplitter(),
    item_tfms=RandomResizedCrop(224, min_scale=0.5),
    batch_tfms=aug_transforms(),
)

##### Function to make the masks compatible with the datablock's y_coordinate.

In [ ]:
# function to get the mask given an image
def get_mask(x):
    # load the numpyfile
    numpyfile = np.load(mask_path / f"{x.stem}.npz")
    # access the data stored in the compressed file format
    data = numpyfile.f.arr_0
    # close the file as to save memory
    numpyfile.close()
    # return the extracted mask
    return data

##### Setup check for GPU usage with PyTorch and CUDA
##### defining the DataLoader accordingly

In [ ]:
# if pytorch and cuda is setup correctly enable the dataloader to run on the gpu by passing it the cuda device (Installation of cuda https://developer.nvidia.com/cuda-downloads is necessary)
if torch.cuda.is_available():
    dataloader = satellite_block.dataloaders(
        image_path, bs=32, device=torch.device("cuda")
    )
    
else:
    dataloader = satellite_block.dataloaders(image_path, bs=32)

### Training process

#### Creation of a UNet-Learner for segmentation of images
##### dice_wo_bg, Dice: metrics to evaluate performance
##### Self_attention=True:
##### act_cls=Mish:
##### opt_func=ranger

##### 3 training sessions to the final model

In [ ]:
# define the unet learner, the metrics, also enable self attention, set mish as the activation function and pass ranger as the optimization function
learner = unet_learner(
    dataloader,
    resnet34,
    metrics=[dice_wo_bg, Dice],
    self_attention=True,
    act_cls=Mish,
    opt_func=ranger,
)

#### First iteration of model training:
##### - derive optimal learning rate
##### - maximum of 20 epochs
##### - Early stopping based on dice_wo_bg value

In [ ]:
# first pass model training, with early stopping after seeing no improvment in the custom defined metric for 4 epochs, train max of 20 epochs
learner.fit_one_cycle(
    20,
    lr_max=lr.valley,
    cbs=[
        EarlyStoppingCallback(
            monitor="dice_wo_bg", min_delta=0.01, patience=4, comp=np.greater
        ),
        SaveModelCallback(
            monitor="dice_wo_bg", min_delta=0.01, comp=np.greater, fname="first_pass"
        ),
    ],
)

In [ ]:
# load the learner and show some examples
learner = learner.load("first_pass")
learner.show_results(max_n=5)

In [1]:
Results First Pass

SyntaxError: invalid syntax (<ipython-input-1-a6570acf0659>, line 1)

In [ ]:
# load the model and show some examples
learner = learner.load("second_pass")
learner.show_results(max_n=5)

Results Second Pass

In [ ]:
# load the model and show some examples
learner = learner.load("finetuned_model")
learner.show_results(max_n=5)

Results Third Pass